Universidade Federal do Rio Grande do Norte

Ciência de Dados - 2020.6

Alteração do Jupyter NoteBook original de Andressa Stéfany

# Dash App com Localização de Aldeias Indígenas Brasileiras

Este Jupyter Notebook foi criado para apresentação acadêmica em turma de Ciência de Dados e foi desenvolvido em cima do material passado por Andressa Stéfany. As informações a respeito das tecnologias usadas foram citadas de forma literal mantendo, inclusive, o idioma.

O projeto desenvolvido gera um aplicativo Dash mostrando informações a respeito da localização geoespacial de aldeias indígenas brasileiras.

>Dash is a productive Python framework for building web analytic applications.
>
>Written on top of Flask, Plotly.js, and React.js, Dash is ideal for building data visualization apps with highly custom user interfaces in pure Python. It's particularly suited for anyone who works with data in Python.
>
> For more informations [click here](https://dash.plotly.com/introduction).

## Importação dos Módulos

In [ ]:
# Instalar Plotly
!pip install Plotly==4.12

# Instalar Dash
!pip install dash
!pip install dash-html-components
!pip install dash-core-components

# Instalar Geopandas
!pip install geopandas

In [ ]:
import os.path
import sys, json
import requests
import subprocess
import requests

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import geopandas as gp

from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

## Configuração do Ngrok

Caso esteja usando o Google Colab ou outro ambiente remoto, será preciso utilizar o Ngrok para acessar a aplicação. Se estiver rodando localmente, pule essa seção e os procedimentos utilizando o Ngrok.

>[Ngrok](https://ngrok.com/) will be used to create the Dash app URL.
>
>First, you need to download of Ngrok. The function below downloads and starts the external URL creation process:

In [ ]:
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [ ]:
Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

## Adquirindo os Dados

O IBGE disponibilizou dados geográficos de localidades brasileiras de acordo com pesquisa feita em 2010. Dentre os locais, foram classificados como "ALDEIA INDÍGENA" os pontos que possuem "casa ou conjunto de casas ou malocas [...] que serve de habitação para o indígena e aloja diversas famílias" e com "no mínimo, 20 habitantes indígenas em uma ou mais moradias".

A publicação oficial do instituto pode ser acessada clicando [aqui](https://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14126-asi-ibge-disponibiliza-coordenadas-e-altitudes-para-21304-localidades-brasileiras). Os dados utilizados e as informações de classificação das localidades podem ser acessadas clicando [aqui](ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades).

Nesse projeto, foi utilizado o formato ShapeFile oferecido pelo instituto. Usaremos o GeoPandas para processar os arquivos que podem ser acessados também na pasta *data* deste repositório.

In [ ]:
# Caso esteja usando o Colab, faça upload das bases de dados (.prj, .shp, .shx e .dbf) e mova para a pasta data
!mkdir data

In [18]:
# Inicialização do GeoDataFrame
df = gp.read_file('data/BR_Localidades_2010_v1.shp')

Nosso GeoDataFrame possui localidades de várias categorias, nesse estudo usaremos apenas as localizações marcadas como "ALDEIA INDIGENA".

In [19]:
df.NM_CATEGOR.unique()

array(['CIDADE', 'VILA', 'POVOADO', 'ALDEIA INDÍGENA', 'NÚCLEO',
       'PROJETO DE ASSENTAMENTO', 'LUGAREJO', 'AUI'], dtype=object)

In [20]:
df = df[df.NM_CATEGOR == 'ALDEIA INDÍGENA'].reset_index()

Será preciso conseguir mais informações a respeito de cada localidade para criar os filtros do Dash App. Usaremos a API do IBGE para conseguir esses daddos.

In [21]:
response = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/estados')
ufs = {uf['nome'].upper(): {'name': uf['nome'], 'initial': uf['sigla'], 'region': {'name': uf['regiao']['nome'], 'initial': uf['regiao']['sigla']}} for uf in response.json()}

Agora, é preciso juntar as novas informações com o DataFrame que já possuíamos.

In [22]:
df['UF_SIGLA'] = df['NM_UF'].apply(lambda uf: ufs.get(uf)['initial'])
df['NM_REGIAO'] = df['NM_UF'].apply(lambda uf: ufs.get(uf)['region']['name'])
df['REGIAO_SIGLA'] = df['NM_UF'].apply(lambda uf: ufs.get(uf)['region']['initial'])
df['UF'] = df['NM_UF'].apply(lambda uf: ufs.get(uf)['name'])

Para não precisarmos refazer este processo toda vez que iniciarmos o aplicativo, vamos salvar um arquivo csv com as informações pertinentes. Desta forma, nem mesmo o GeoPandas será necessário para execução do aplicativo.

In [24]:
df[['ID', 'NM_LOCALID', 'LONG', 'LAT', 'ALT', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA', 'UF']].to_csv('data/aldeias_indigenas.csv')

## Prévia dos Gráficos

Antes de montarmos o aplicativo Dash, primeiro vamos formular e estilizar os gráficos usando o CSV que contruímos.

In [26]:
# Inicialização do DataFrame do pandas
df = pd.read_csv('data/aldeias_indigenas.csv')

O primeiro gráfico é um mapa com pontos que representam as localizações espaciais das aldeias indígenas. O posicionamento do ponto é definido por longitude e latitude e a cor pela altitude. Utilizaremos o módulo **plotly express** para gerar um gráfico do tipo Scatter MapBox.

In [29]:
fig_map = px.scatter_mapbox(df, 
                            lat='LAT', 
                            lon='LONG', 
                            color='ALT',
                            text='NM_LOCALID',
                            hover_data=['UF', 'NM_REGIAO'],
                            labels={'NM_LOCALID':'Nome', 'LAT':'Latitude', 'LONG':'Longitude', 'ALT':'Altitude', 'UF':'Estado', 'NM_REGIAO': 'Região'},
                            zoom=2.8,
                            mapbox_style='carto-positron')

fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, coloraxis_colorbar={'xanchor':'right', 'x':1})

O segundo, é um gráfico de barras que mostra a quantidade de aldeias por estado. Então, vamos criar um novo DataFrame a partir da soma do agrupamento das ocorrências por estado, sigla, região e sigla da região.

In [30]:
# Inicialização de DataFrame secundário
count_df = df[['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA', 'ID']].groupby(['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA']).count().reset_index(level=['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA'])
count_df.rename(columns={'ID': 'ALDEIAS'}, inplace=True)

Para renderizar o gráfico, usaremos o módulo **plotly graph objects**.

In [31]:
fig_bar = go.Figure(data=[go.Bar(
                x=count_df['UF_SIGLA'],
                y=count_df['ALDEIAS'],
                text=count_df['ALDEIAS'],
                textposition='auto')])

fig_bar.update_layout(showlegend=False,
                    margin={'r':0,'t':0,'l':0,'b':0},
                    yaxis={'visible': False, 'showticklabels': False})

## Aplicativo Dash

>Dash apps are composed of [layout](https://dash.plotly.com/layout), it describes what the application looks like, and [interactivity](https://dash.plotly.com/basic-callbacks) of the application.
>
>For layout, We will use the `dash_core_components` and the `dash_html_components` library. But you can use also build your own with JavaScript and React.js.
>
>
>- The layout is composed of componenets like `html.Div` and `dcc.Graph`.
- The `dash_core_components` library has components for every HTML tag. For example: Div, H6 and Br.
- The `dash_html_components`library describe components that are interactive. For example: Graph, Input and Slider.

### Versão inicial

Agora, vamos juntar o código apresentado nas prévias com a implementação de um aplicativo dash bem básico.

In [33]:
%%writefile basic_dash_app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', 'https://fonts.googleapis.com/css2?family=Quicksand&display=swap']

app = dash.Dash(__name__,
    external_stylesheets=external_stylesheets,
    title='Aldeias Indígenas',
    update_title=None
)

server = app.server

df = pd.read_csv('data/aldeias_indigenas.csv')

fig_map = px.scatter_mapbox(df, 
                            lat='LAT', 
                            lon='LONG', 
                            color='ALT',
                            text='NM_LOCALID',
                            hover_data=['UF', 'NM_REGIAO'],
                            labels={'NM_LOCALID':'Nome', 'LAT':'Latitude', 'LONG':'Longitude', 'ALT':'Altitude', 'UF':'Estado', 'NM_REGIAO': 'Região'},
                            zoom=2.8,
                            mapbox_style='carto-positron')

fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, coloraxis_colorbar={'xanchor':'right', 'x':1})

count_df = df[['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA', 'ID']].groupby(['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA']).count().reset_index(level=['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA'])
count_df.rename(columns={'ID': 'ALDEIAS'}, inplace=True)

fig_bar = go.Figure(data=[go.Bar(
                x=count_df['UF_SIGLA'],
                y=count_df['ALDEIAS'],
                text=count_df['ALDEIAS'],
                textposition='auto')])

fig_bar.update_layout(showlegend=False,
                    margin={'r':0,'t':0,'l':0,'b':0},
                    yaxis={'visible': False, 'showticklabels': False})

app.layout = html.Div([                
    dcc.Graph(
        id='map',
        figure=fig_map
    ),
    dcc.Graph(
        id='bar',
        figure=fig_bar
    )
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Overwriting basic_dash_app.py



Executando o arquivo por meio do Ngrok...

In [34]:
download_ngrok()

--2020-11-20 03:06:50--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.0.105.155, 34.198.20.103, 54.161.19.10, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.0.105.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  12.8MB/s    in 1.0s    

2020-11-20 03:06:51 (12.8 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [35]:
retorno = get_tunnel()
print(retorno)
!python basic_dash_app.py

Response(url='https://7353fba53a92.ngrok.io', error=None)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "basic_dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
^C


Enquanto a célula acima estiver rodando, podemos visualizar o aplicativo pela url do Ngrok.

### Interação com o Usuário e Estilização

O Dash possibilita interações com o usuário por meio de callback. É interessante consultar a [documentação da ferramenta](https://dash.plotly.com/basic-callbacks) para visualizar mais informações.

Neste projeto, as interações se resumem a filtrar as localizações que serão mostradas no gráficos de acordo com a Unidade Federativa e Região. Os componentes gráficos Dropdown do módulo **dash_core_components** possibilitam essa escolha.

A estilização do aplicativo final feita inteiramente por CSS disponibilizado na pasta *assets* deste repositório.

In [36]:
# Caso esteja usando o Colab, é preciso criar a pasta assets e fazer o upload dos arquivos dela (styles.css e github.svg)
!mkdir assets

### Versão Final

O código da versão final da aplicação é definido abaixo e contém informações a respeito da fonte dos dados utilizados, os gráficos e componentes de seleção dos filtros.

In [37]:
%%writefile app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from dash.dependencies import Input, Output

# importação das folhas de estilo externas
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css', 'https://fonts.googleapis.com/css2?family=Quicksand&display=swap']

# inicialização do objeto Dash e definição do título da página
app = dash.Dash(__name__,
    external_stylesheets=external_stylesheets,
    title='Aldeias Indígenas',
    update_title=None
)

# variável utilizada apenas no deploy
server = app.server

# criação do DataFrame a partir dos dados pre-processados
df = pd.read_csv('data/aldeias_indigenas.csv')

# regiões e estados por regiões, definidas para uso nos filtros
regions = df.NM_REGIAO.unique()
states_by_region = {region: df[df.NM_REGIAO == region].UF.unique() for region in regions}

# definição do layout do aplicativo
app.layout = html.Div([

    # barra de navegação superior da página com link para repositório
    html.Div([

        html.H1('Aldeias Indígenas no Brasil'),
        html.A([
            html.Img(src=app.get_asset_url('github.svg'))
        ], href='https://github.com/mmartiniano/brazilian-indigenous-village-location', target='_blank')

    ], className='navbar'),

    html.Div([

        html.Div([

            html.H1('Localização Geográfica das Aldeias Indígenas Brasileiras'),

            html.P('Selecione as localidades para filtrar os resultados:'),

            # componente de escolha de regiões
            dcc.Dropdown(
                id='regions',
                options=[{'label': region, 'value': region} for region in regions],
                placeholder='Selecione regiões',
                multi=True,
            ),

            # componente de escolha de estados
            dcc.Dropdown(
                id='states',
                placeholder='Selecione estados',
                multi=True,
            ),

            # informações das fontes
            html.P('''
                    O IBGE disponibilizou dados geográficos de localidades brasileiras
                    de acordo com pesquisa feita em 2010. Dentre os locais, foram classificados
                    como "ALDEIA INDÍGENA" os pontos que possuem "casa ou conjunto de casas ou
                    malocas [...] que serve de habitação para o
                    indígena e aloja diversas famílias" e com "no mínimo, 20 habitantes indígenas em
                    uma ou mais moradias".                     
            '''),
            html.P(['A publicação oficial do instituto pode ser acessada clicando ',
                html.A('aqui', target='_blank', href='https://agenciadenoticias.ibge.gov.br/agencia-sala-de-imprensa/2013-agencia-de-noticias/releases/14126-asi-ibge-disponibiliza-coordenadas-e-altitudes-para-21304-localidades-brasileiras'),
                '''. A localização geográfica em longitude, latitude e altitude das aldeias podem
                ser visualizadas no mapa. Além disso, o gráfico de barras mostra a
                quantidade de aldeias por região.
                Os dados utilizados e as informações de classificação das localidades podem ser acessadas clicando
                ''',
                html.A('aqui', target='blank', href='ftp://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/localidades'),
                '.'
            ])

        ], className='left'),

        # gráficos
        html.Div([

            dcc.Graph(
                id='map',
                className='map'
            ),

            dcc.Graph(
                id='bar',
                className='bar'
            )

        ], className='right')

    ], className='main')
                 
])

# função chamada quando ocorre mudança no dropdown de região
# muda as opções do dropdown de estados de acordo com a região
@app.callback(
    Output('states', 'options'),
    [Input('regions', 'value')])
def set_states_options(selected_regions):
    if selected_regions is None or len(selected_regions) <= 0:
        selected_regions = regions[:]

    return [{'label': state, 'value': state} for region in selected_regions for state in states_by_region[region]]

# função executda sempre que há mudança nos filtros
# atualiza os gráficos de acordo com as escolhas de estado e região
@app.callback([
    Output('map', 'figure'),
    Output('bar', 'figure')
    ],[
    Input('regions', 'value'),
    Input('states', 'value')
    ]
)
def update_graph(selected_regions, selected_states) :

    if selected_regions is None or len(selected_regions) <= 0:
        selected_regions = regions[:]
    
    if selected_states is None or len(selected_states) <= 0:
        selected_states = df.UF.unique()

    fig_map = px.scatter_mapbox(df[df.NM_REGIAO.isin(selected_regions) & df.UF.isin(selected_states)], 
                            lat='LAT', 
                            lon='LONG', 
                            color='ALT',
                            text='NM_LOCALID',
                            hover_data=['UF', 'NM_REGIAO'],
                            labels={'NM_LOCALID':'Nome', 'LAT':'Latitude', 'LONG':'Longitude', 'ALT':'Altitude', 'UF':'Estado', 'NM_REGIAO': 'Região'},
                            zoom=2.8,
                            mapbox_style='carto-positron')

    fig_map.update_layout(margin={'r':0,'t':0,'l':0,'b':0}, coloraxis_colorbar={'xanchor':'right', 'x':1})

    count_df = df[['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA', 'ID']].groupby(['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA']).count().reset_index(level=['UF', 'UF_SIGLA', 'NM_REGIAO', 'REGIAO_SIGLA'])
    count_df.rename(columns={'ID': 'ALDEIAS'}, inplace=True)

    count_df = count_df[count_df.NM_REGIAO.isin(selected_regions) & count_df.UF.isin(selected_states)]

    fig_bar = go.Figure(data=[go.Bar(
                x=count_df['UF_SIGLA'],
                y=count_df['ALDEIAS'],
                text=count_df['ALDEIAS'],
                textposition='auto')])

    fig_bar.update_layout(showlegend=False,
                    margin={'r':0,'t':0,'l':0,'b':0},
                    yaxis={'visible': False, 'showticklabels': False})

    return fig_map, fig_bar


if __name__ == '__main__':
    app.run_server(debug=True)

Writing app.py


Caso não tenha baixado o Ngrok na execução da versão incial, execute:

In [ ]:
download_ngrok()

Enquanto a célula abaixo estiver sendo executada, a versão final do aplicativo pode ser visualizada através do Ngrok.

In [38]:
retorno = get_tunnel()
print(retorno)
!python app.py

Response(url='https://83f0d3025ff1.ngrok.io', error=None)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


## Autoria

Desenvolvido por [Lucas Miguel](https://github.com/mmartiniano) a partir de material ministrado por Andressa Stéfany.

Código em [repositório](https://github.com/mmartiniano/brazilian-indigenous-village-location) público disponível no Github.